In [1]:
#importing required libraries
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

import collections
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize 

In [71]:
#reading csv files and filling the NaN values
if os.path.isfile('users.csv'):
    users = pd.read_csv("users.csv", nrows=22000)
    apps = pd.read_csv("apps.csv")
else:
    print("get users.csv from drive or run the previous notebook")

In [72]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 10 columns):
user_id                22000 non-null int64
prefered_location      22000 non-null object
user_education         22000 non-null object
user_skills            22000 non-null object
user_education_year    22000 non-null object
work_history_count     22000 non-null int64
user_experience        22000 non-null int64
currently_employed     22000 non-null object
managed_others         22000 non-null object
managed_howmany        22000 non-null int64
dtypes: int64(4), object(6)
memory usage: 1.7+ MB


In [73]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 14 columns):
company                22000 non-null object
education              22000 non-null object
experience             22000 non-null object
industry               22000 non-null object
jobdescription         22000 non-null object
jobid                  22000 non-null int64
joblocation_address    22000 non-null object
jobtitle               22000 non-null object
numberofpositions      21999 non-null float64
payrate                22000 non-null object
postdate               22000 non-null object
site_name              13063 non-null object
skills                 22000 non-null object
uniq_id                22000 non-null object
dtypes: float64(1), int64(1), object(12)
memory usage: 2.3+ MB


In [74]:
#dropping outliers


# users.drop(users.loc[users['ManagedHowMany'] > 5].index, inplace=True)
# users.drop(users.loc[users['TotalYearsExperience'] >=30].index, inplace=True)
# users.drop(users.loc[users['WorkHistoryCount'] > 8].index, inplace=True)

In [75]:
apps.columns

Index([u'company', u'education', u'experience', u'industry', u'jobdescription',
       u'jobid', u'joblocation_address', u'jobtitle', u'numberofpositions',
       u'payrate', u'postdate', u'site_name', u'skills', u'uniq_id'],
      dtype='object')

In [76]:
users.columns

Index([u'user_id', u'prefered_location', u'user_education', u'user_skills',
       u'user_education_year', u'work_history_count', u'user_experience',
       u'currently_employed', u'managed_others', u'managed_howmany'],
      dtype='object')

In [77]:
users.shape

(22000, 10)

In [78]:
apps.shape

(22000, 14)

In [79]:
users.head(2)

,user_id,prefered_location,user_education,user_skills,user_education_year,work_history_count,user_experience,currently_employed,managed_others,managed_howmany
0,47,chennai,mechanical engineering,ites,1999,3,10,yes,no,0
1,80,bengaluru,electrical electronics engineering,it software to application programming,1985,5,11,yes,yes,5


In [80]:
apps.head(2)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,mm media pvt ltd,ug b tech b e to any specialization pg any ...,0 to 1 years,media entertainment internet,job description â ownend me jobs ...,210516002263,chennai,walkin data entry operator night shift,NaN,150k to 225k p a,2016-05-21 19:30:00 +0000,NaN,ites,43b19632647068535437c774b6ca6cf8
1,find live infotech,ug b tech b e to any specialization pg mba ...,0 to 0 years,advertising pr mr event management,job description â ownend me jobs ...,210516002391,chennai,work based onhome based part time,60.0,150k to 250k p a 20k,2016-05-21 19:30:00 +0000,NaN,marketing,d4c72325e57f89f364812b5ed5a795f0


In [81]:
#removing the unncessary rows
del apps['site_name']
del apps['jobid']
del apps['industry']


In [82]:
print(apps['jobtitle'].describe())

count                            22000
unique                           17148
top       business development manager
freq                               101
Name: jobtitle, dtype: object


In [83]:
print(apps['jobtitle'].unique())

['walkin data entry operator  night shift '
 'work based onhome based part time ' 'pl sql developer  to sql' ...
 'full stack development engineer'
 'sr ui developer  technical lead  to html  css  javascript  angularjs'
 'rf firmware engineers']


In [84]:
import collections
job_type_count = collections.Counter(list(apps['jobtitle']))
job_type_count.most_common(10)

[('business development manager', 101),
 ('business development executive', 96),
 ('software engineer', 88),
 ('android developer', 77),
 ('php developer', 72),
 ('project manager', 71),
 ('web designer', 69),
 ('content writer', 68),
 ('java developer', 61),
 ('senior software engineer', 60)]

In [85]:
job_type_count = collections.Counter(list(apps['uniq_id']))
job_type_count.most_common(10)

[('8e6baca57b00dd028ee71e509ebbd517', 1),
 ('acf1d15d447eb784d80264c7bf93ad53', 1),
 ('8ebddbdbc4977f0c9aaf30057f0fd060', 1),
 ('84aa071a193771c7d02d9b86cd2c3631', 1),
 ('35bfb62d31ec55ba3cd8111385c79971', 1),
 ('503f7285d325e4ce25dd8dbbd981ca4b', 1),
 ('f964843fd56bc1cd0e5dfa8edcb2a05f', 1),
 ('4263b7cb2ab24a6c1167b1eaefb7a97e', 1),
 ('4cb81524477c6e235573b3b4ada0e6db', 1),
 ('5bff19e38756598af7617dcaad86d6dd', 1)]

In [86]:
print(apps['joblocation_address'].describe())

count                   22000
unique                   2317
top       bengaluru bangalore
freq                     3802
Name: joblocation_address, dtype: object


In [87]:
print(apps['joblocation_address'].describe())

count                   22000
unique                   2317
top       bengaluru bangalore
freq                     3802
Name: joblocation_address, dtype: object


In [88]:
#data preprocessing steps

import re
from nltk.tokenize import sent_tokenize, word_tokenize 
stop_words = stopwords.words("english")

def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("yrs", " years")\
                           .replace("-", " to").replace("/", " ").replace(",", "")
        
                            
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
#     porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    

               
    
    return x 

In [89]:
# applying data preprocessing
users["prefered_location"] = users["prefered_location"].fillna("").apply(preprocess)
users["user_education"] = users["user_education"].fillna("").apply(preprocess)
users["user_skills"] = users["user_skills"].fillna("").apply(preprocess)
users["user_education_year"] = users["user_education_year"].fillna("").apply(preprocess)
users["user_experience"] = users["user_experience"].fillna("").apply(preprocess)
users["currently_employed"] = users["currently_employed"].fillna("").apply(preprocess)



apps["company"] = apps["company"].fillna("").apply(preprocess)
apps["experience"] = apps["experience"].fillna("").apply(preprocess)
apps["education"] = apps["education"].fillna("").apply(preprocess)
apps["jobdescription"] = apps["jobdescription"].fillna("").apply(preprocess)
apps["joblocation_address"] = apps["joblocation_address"].fillna("").apply(preprocess)
apps["jobtitle"] = apps["jobtitle"].fillna("").apply(preprocess)
apps["payrate"] = apps["payrate"].fillna("").apply(preprocess)
apps["skills"] = apps["skills"].fillna("").apply(preprocess)

In [90]:
def againpreprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
            .replace("ug", "under graduate").replace("pg", "post graduate")\
            .replace("b tech", "bachelor of technology").replace("b e", "bachelor of engineering")\
            .replace("p a", "per annum").replace("mca", "master of computer applications")
                           
    return x

In [91]:
apps["education"] = apps["education"].fillna("").apply(againpreprocess)
apps["payrate"] = apps["payrate"].fillna("").apply(againpreprocess)

In [92]:
# stopwords removal
stop = stopwords.words('english')

users['user_skills'] = users['user_skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

apps['jobdescription'] = apps['jobdescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
apps['jobtitle'] = apps['jobtitle'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
apps['skills'] = apps['skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
# apps['experience'] = apps['experience'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
apps['education'] = apps['education'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
apps['payrate'] = apps['payrate'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

In [93]:
#lemmatization

from textblob import TextBlob
from textblob import Word

apps['education'] = apps['education'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
apps['experience'] = apps['experience'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
apps['jobdescription'] = apps['jobdescription'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
apps['jobtitle'] = apps['jobtitle'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
apps['payrate'] = apps['payrate'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
apps['skills'] = apps['skills'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

askills_vectorizer = CountVectorizer()
askills_features   = askills_vectorizer.fit_transform(apps['skills'])

uskills_vectorizer = CountVectorizer()
uskills_features   = uskills_vectorizer.fit_transform(users['user_skills'])

alocation_vectorizer = CountVectorizer()
alocation_features   = alocation_vectorizer.fit_transform(apps['joblocation_address'])

ulocation_vectorizer = CountVectorizer()
ulocation_features   = ulocation_vectorizer.fit_transform(users['prefered_location'])



extra_features = hstack((askills_features, uskills_features, alocation_features, ulocation_features)).tocsr()

extra_features.get_shape()

(22000, 861)

In [96]:
def bag_of_words_model(job_id, num_results):
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(extra_features,extra_features[job_id])
    
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]

    #data frame indices of the 9 smallest distace's
    users_df_indices = list(users.index[indices])
    
    

    
    
    for i in range(0,len(indices)):
        print('User ID :',users['user_id'].loc[users_df_indices[i]])
        print ('User Skills:', users['user_skills'].loc[users_df_indices[i]])
        print ('User Location:', users['prefered_location'].loc[users_df_indices[i]])
        print ('Currently employed:', users['currently_employed'].loc[users_df_indices[i]])
        print ('User education:', users['user_education'].loc[users_df_indices[i]])
        print ('Euclidean similarity with the query :', pdists[i])
        print('='*60)

#call the bag-of-words model for a product to get similar products.
bag_of_words_model(0, 20) # change the index if you want to.

('User ID :', 47)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
('User education:', 'mechanical engineering')
('Euclidean similarity with the query :', 0.0)
('User ID :', 115402)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
('User education:', 'information technology')
('Euclidean similarity with the query :', 0.0)
('User ID :', 118041)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'no')
('User education:', 'electrical electronics engineering')
('Euclidean similarity with the query :', 0.0)
('User ID :', 120023)
('User Skills:', 'ites')
('User Location:', 'chennai  cuddalore')
('Currently employed:', 'yes')
('User education:', 'master of science')
('Euclidean similarity with the query :', 1.0)
('User ID :', 118043)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'no')
('User education:', 'information technology')
('Euclidean similarity with th

In [97]:
# tfidf similarity based approach

tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_atitle_features = tfidf_title_vectorizer.fit_transform(apps['jobtitle'])
tfidf_aeducation_features = tfidf_title_vectorizer.fit_transform(apps['education'])
tfidf_ueducation_features = tfidf_title_vectorizer.fit_transform(users['user_education'])
tfidf_askills_features = tfidf_title_vectorizer.fit_transform(apps['skills'])
tfidf_uskills_features = tfidf_title_vectorizer.fit_transform(users['user_skills'])
# tfidf_uexperience_features = tfidf_title_vectorizer.fit_transform(data['users_experience'])
tfidf_aexperience_features = tfidf_title_vectorizer.fit_transform(apps['experience'])
tfidf_uemployed_features = tfidf_title_vectorizer.fit_transform(users['currently_employed'])
tfidf_ulocation_features = tfidf_title_vectorizer.fit_transform(users['prefered_location'])
tfidf_alocation_features = tfidf_title_vectorizer.fit_transform(apps['joblocation_address'])

In [98]:
tfidf_all_features = hstack((tfidf_atitle_features, tfidf_aeducation_features, tfidf_ueducation_features,
                            tfidf_askills_features,tfidf_uskills_features,tfidf_aexperience_features,
                            tfidf_uemployed_features,tfidf_ulocation_features,tfidf_alocation_features)).tocsr()

tfidf_all_features.get_shape()

(22000, 8387)

In [100]:
def tfidf_model(job_id, num_results):
    # job_id: apparel's id in given corpus
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(tfidf_all_features,tfidf_all_features[job_id])

    # np.argsort will return indices of 9 smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the 9 smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]

    #data frame indices of the 9 smallest distace's
    users_df_indices = list(users.index[indices])
    

    for i in range(0,len(indices)):
        print('User ID :',users['user_id'].loc[users_df_indices[i]])
        print ('User Skills:', users['user_skills'].loc[users_df_indices[i]])
        print ('User Location:', users['prefered_location'].loc[users_df_indices[i]])
        print ('Currently employed:', users['currently_employed'].loc[users_df_indices[i]])
        print ('User education:', users['user_education'].loc[users_df_indices[i]])
        print ('Euclidean similarity with the query image :', pdists[i])
        print('='*64)
tfidf_model(0, 20)
# in the output heat map each value represents the tfidf values of the label word, the color represents the intersection with inputs title

('User ID :', 47)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
('User education:', 'mechanical engineering')
('Euclidean similarity with the query image :', 0.0)
('User ID :', 119313)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
('User education:', 'mechanical engineering')
('Euclidean similarity with the query image :', 2.2020652849791937)
('User ID :', 117796)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
('User education:', 'mechanical engineering')
('Euclidean similarity with the query image :', 2.2637614440758607)
('User ID :', 111580)
('User Skills:', 'ites')
('User Location:', 'bengaluru bangalore  pune  chennai')
('Currently employed:', 'yes')
('User education:', 'civil engineering')
('Euclidean similarity with the query image :', 2.3129265752222623)
('User ID :', 115402)
('User Skills:', 'ites')
('User Location:', 'chennai')
('Currently employed:', 'yes')
(